In [92]:
# install needed libraries
# pip install beautifulsoup4
# pip install requests
# !pip install selenium
# !pip install webdriver-manager
# pip install lxml
# from selenium.webdriver.common.by import By
# pip install lxml

In [103]:
# import libraries
# from bs4 import BeautifulSoup
# import requests 
import csv
import json
import re
import sys
import datetime
import requests
import os
import time
from bs4 import BeautifulSoup
from pandas.core.frame import DataFrame
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import numpy as np
from selenium.common.exceptions import NoSuchElementException

options = webdriver.ChromeOptions()
# 此步骤很重要，设置为开发者模式，防止被各大网站识别出来使用了Selenium
options.add_experimental_option('excludeSwitches', ['enable-automation']) 
chrome_options = Options()
chrome_options.add_argument("--window-size=1920,1080")
#driver = Chrome(chrome_options=chrome_options)
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
# driver.get("https://www.google.com")
driver.maximize_window()

/var/folders/my/bpgmln1s4sv1x1tmmw1b796c0000gn/T/ipykernel_5800/2950689678.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
/var/folders/my/bpgmln1s4sv1x1tmmw1b796c0000gn/T/ipykernel_5800/2950689678.py:30: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


In [65]:
course_title = []
difficulty_level = []
duration = []
skills = []
instructors = []
description = []
rating = []
review_date = []
user_name = []
reviews = []
#there are 84 pages of programs
'''main_url = "https://www.coursera.org/courses?page=2&index=prod_all_products_term_optimization"
driver.get(main_url)
links_in_main_url = driver.find_elements(By.XPATH, "//div[@class='css-1j8ushu']/div/a") # get url of all programs/certificate in main url of coursera
links_in_main_url = [elem.get_attribute('href') for elem in links_in_main_url] # change html to list
for link_for_prog in links_in_main_url: # click in each program url and find corresponding courses url to explore
    driver.get(link_for_prog)
    links_for_courses = driver.find_elements(By.XPATH, "//div/a[@data-e2e='course-link']")#.click()
    links_in_prog_url = [elem.get_attribute('href') for elem in links_for_courses]'''
main_url = 'https://www.coursera.org/sitemap~www~courses.xml'
driver.get(main_url)
source = requests.get(main_url).text
soup = BeautifulSoup(source, 'lxml')
links_in_prog_url = [site.text for site in soup.find_all('loc')] # list of links for courses
for i in range(len(links_in_prog_url)): # run from where we left
    if links_in_prog_url[i] == 'https://www.coursera.org/learn/google-workspace-security':
        link_index = i
        break
links_in_prog_url = links_in_prog_url[link_index:]
for course_link in links_in_prog_url: # click in each course url of each program to get useful information
    driver.get(course_link)
    course_title_find_element = driver.find_elements(By.XPATH, "//div/h1[@class='banner-title banner-title-without--subtitle m-b-0']")
    #banner-title banner-title-without--subtitle m-b-0
    if len(course_title_find_element) == 0:
        course_title_need_to_append = np.nan
    else:
        course_title_need_to_append = course_title_find_element[0].text
    # find all possible ways to find difficulty level and append it/NO Level Specified to the list 
    list_levels_a = list(set([a.text for a in driver.find_elements(By.XPATH, "//div/div[@class='_16ni8zai m-b-0 m-t-1s']") if 'Level' in a.text])) # one possible way to get diff level
    list_levels_b = list(set([b.text for b in driver.find_elements(By.XPATH, "//div/div[@class='_16ni8zai m-b-0']") if 'Level' in b.text])) # another possible way to get diff level
    diff_level_all_possible = list(set(list_levels_a + list_levels_b)) # find level related info and append in diff level
    if len(diff_level_all_possible) == 0:
        difficulty_level_need_to_append = np.nan
    else:
        difficulty_level_need_to_append = diff_level_all_possible[0]
    # similar to diff_level, find all possible ways to find duration and append it/NO Duarion Specified to the list
    time_list = list(set([a.text for a in driver.find_elements(By.XPATH, "//div/div[@class='_16ni8zai m-b-0 m-t-1s']/span") if 'complete' in a.text]))
    if len(time_list) == 0:
        duration_need_to_append = np.nan
    else:
        duration_need_to_append = time_list[0]          
    # for all skills in course page, create a list and append to skills list
    # one course could have serveral skills
    List_skills_per_course = driver.find_elements(By.XPATH, "//li/span/span/span[@class='_ontdeqt']")
    skills_need_to_append = [a.text for a in List_skills_per_course]
    # similar to skills in course page, create a list of instructors in each course and append to instructors list
    # one course could have serveral instructors and some of them could not be name of instructor but certificate (such as google cetificate)
    List_instructors_per_course = driver.find_elements(By.XPATH, "//div/h3[@class='instructor-name headline-3-text bold']")
    instructors_need_to_append = [a.text for a in List_instructors_per_course]
    # one course has only one description but one description might be divided into multi parts in source code
    # so we create a list of descriptions in each course and paste them together appending to instructors list
    description_list = [a.text for a in driver.find_elements(By.XPATH, "//div/div/div[@class='content']/div[@class='content-inner']/p")]
    description_need_to_append = ' '.join(description_list)
    # since all reviews are in a link on course page, we need to click on it first
    # we only scraped at most 25 top helpful reviews
    # check if reviews link is clickable
    if len(driver.find_elements(By.PARTIAL_LINK_TEXT, 'reviews')) != 0:
        prop = driver.find_element(By.PARTIAL_LINK_TEXT, 'reviews').get_property('disabled') 
        if prop:
            driver.find_element(By.PARTIAL_LINK_TEXT, 'reviews').click()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            review_list_a = soup.find_all(class_ = 'cds-63 css-1cyk8pe cds-65 cds-grid-item cds-72') # possible ways for finding reviews
            review_list_b = soup.find_all(class_ = 'cds-31 css-1cyk8pe cds-33 cds-grid-item cds-40')
            review_list_c = soup.find_all(class_ = 'cds-63 css-1ktinjm cds-65 cds-grid-item cds-134')
            review_list = review_list_a
            if len(review_list_a) == 0:
                review_list = review_list_c
                if len(review_list_c) ==0:
                    review_list = review_list_b
            driver.switch_to.window(driver.window_handles[-1]) # switch to most recent active window (reviews)
            reviews_list = driver.find_elements(By.XPATH, "//div/div[@class='reviewText']/div/div/p[1]")
            if len(review_list) != len(reviews_list):
                continue # if we cannot find accurate rating,etc , then we ignore such course
            for review in reviews_list:
                reviews.append(review.text)      
            # after geting into url of reviews page, we choose up to 25 reviews per course and fill in corresponding info
            for each_review in review_list:
                rating_need_to_append = len(each_review.find_all("title", string='Filled Star'))
                user_name_need_to_append = each_review.find_all(class_ = 'cds-1 reviewerName p-x-1s css-14d8ngk cds-3')[0].text
                review_date_need_to_append = each_review.find_all(class_ = 'cds-1 dateOfReview p-x-1s css-14d8ngk cds-3')[0].text
                user_name.append(user_name_need_to_append)
                review_date.append(review_date_need_to_append)
                rating.append(rating_need_to_append)
                course_title.append(course_title_need_to_append)
                difficulty_level.append(difficulty_level_need_to_append)
                duration.append(duration_need_to_append)
                skills.append(skills_need_to_append)
                instructors.append(instructors_need_to_append)
                description.append(description_need_to_append)
    else:
        reviews.append(np.nan)
        user_name.append(np.nan) # if user_name is np.nan, then the course has no reviews
        review_date.append(np.nan)
        rating.append(np.nan)
        course_title.append(course_title_need_to_append)
        difficulty_level.append(difficulty_level_need_to_append)
        duration.append(duration_need_to_append)
        skills.append(skills_need_to_append)
        instructors.append(instructors_need_to_append)
        description.append(description_need_to_append)
        # since reviews again are JavaScript rendered, we change back to selenium to get reviews
    if len(course_title) != len(reviews):
        print(course_link)
        #print(reviews_list.text)





KeyboardInterrupt: 

In [41]:
len(reviews)

1544

In [42]:
d = {'course_title': course_title, 'difficulty_level': difficulty_level, 'duration': duration, 
    'skills': skills, 'instructors': instructors, 'description':description, 'rating':rating,
    'review_date': review_date, 'user_name': user_name, 'reviews':reviews}
df = pd.DataFrame(data=d)

In [43]:
df.to_csv('/Users/lijiaqi/Documents/usc/2022fall/DSCI560/project_dataset_update_another.csv')

In [45]:
# paste two parts of data together
below = pd.read_csv('/Users/lijiaqi/Documents/usc/2022fall/DSCI560/project_dataset_update_another.csv')  
above = pd.read_csv('/Users/lijiaqi/Documents/usc/2022fall/DSCI560/project_dataset_update.csv')  

In [49]:
newdf = pd.concat([above,below])
# reset multiindex for level 1
newdf.reset_index(inplace=True)

In [51]:
newdf = newdf.drop(['index', 'Unnamed: 0'], axis=1)

In [53]:
# download data as csv
newdf.to_csv('/Users/lijiaqi/Documents/usc/2022fall/DSCI560/proj_whole_data.csv')

Edx Web Scrapper


In [217]:
main_url = 'https://www.udacity.com/courses/all'
source = driver.get(main_url)
source_string = driver.page_source
soup = BeautifulSoup(source_string, 'lxml')
all_courses = soup.find_all(class_ = 'card_container__25DrK')
# find all links corresponding to course_titles in order to find reviews
elems = driver.find_elements(By.XPATH,"//a[@href]")
links = []
for elem in elems:
    if '--' in elem.get_attribute("href"):
        if elem.get_attribute("href") not in links:
            links.append(elem.get_attribute("href"))
links = links[:-3]
i = 0 

In [275]:
# if missing then nan
course_title = [] #
difficulty_level = [] #
duration = [] #
skills = [] #
instructors = [] #
description = [] #
rating = []
review_date = []
user_name = []
reviews = []
main_url = 'https://www.udacity.com/courses/all'
source = driver.get(main_url)
source_string = driver.page_source
soup = BeautifulSoup(source_string, 'lxml')
all_courses = soup.find_all(class_ = 'card_container__25DrK')
# find all links corresponding to course_titles in order to find reviews
'''elems = driver.find_elements(By.XPATH,"//a[@href]")
links = []
for elem in elems:
    if '--' in elem.get_attribute("href"):
        if elem.get_attribute("href") not in links:
            links.append(elem.get_attribute("href"))
links = links[:-3]'''
i = 0 
for course in all_courses: 
    title_to_append = course.find(class_ = 'card_title__35G97').text
    description_to_append = course.find(class_ = 'card_summary__1HlQ7').text
    candidate_level_intermediate = course.find(class_ = 'card_level__2HNxe card_intermediate__prUKS')
    candidate_level_beginner = course.find(class_ = 'card_level__2HNxe card_beginner__1tLPm')
    candidate_level_advanced = course.find(class_ = 'card_level__2HNxe card_advanced__3oCw3')
    if candidate_level_intermediate:
        diff_level_to_append = candidate_level_intermediate.text
    elif candidate_level_beginner:
        diff_level_to_append = candidate_level_beginner.text
    elif candidate_level_advanced:
        diff_level_to_append = candidate_level_advanced.text
    else:
        diff_level_to_append = np.nan
    course_review_url = links[i]
    review_source = driver.get(course_review_url)
    try: #scrape durations
        duration_to_append = driver.find_element(By.XPATH, "//ul[@class='degree-info-columns_columnList__D-UoB']/li/div/h5").text +  driver.find_element(By.XPATH, "//ul[@class='degree-info-columns_columnList__D-UoB']/li/div/p").text
    except NoSuchElementException:  #spelling error making this code not work as expected
        duration_to_append = np.nan
    try: #scrape skills
        List_skills_per_course = driver.find_elements(By.XPATH, "//div/ol/li[@class='degree-syllabus_part__hQ0du']/h5")
        skills_to_append = [a.text for a in List_skills_per_course]    
    except NoSuchElementException:
        try:
            List_skills_per_course = driver.find_elements(By.XPATH, "//a/div/h2[@class='h4']")
            skills_to_append = [a.text for a in List_skills_per_course]
        except NoSuchElementException:
            skills_to_append = [np.nan]
    try:  #scrape instructors      
        instructors_per_course = driver.find_elements(By.XPATH, "//li[@class='card_container__uEUZV card_topRight__1i-5a _brand-refresh_card__NfYin']/article/h3")
        instructors_to_append = [a.text for a in instructors_per_course]
    except NoSuchElementException:
        instructors_to_append = [np.nan]
    # we take up to 5 reviews for udacity, since there is no links for all reviews
    try:
        reviewers_per_course = driver.find_elements(By.XPATH, "//div/div/h5[@class='_card_cardStudent__3i3EG']")
        user_name_to_append = [a.text for a in reviewers_per_course]
        ratings_per_course =  driver.find_elements(By.XPATH, "//div/div/span[@class='rating-stars_visuallyHidden__3U2dC']") 
        reviews_per_course = driver.find_elements(By.XPATH, "//div/div[@class='_card_container__2UvQo']/p[@class='_card_cardReview__2Wcmp']")
        review_date_to_append = [np.nan for a in reviewers_per_course]
        ratings_to_append = [a.text for a in ratings_per_course][1:]
        reviews_to_append = [a.text for a in reviews_per_course]
    except NoSuchElementException:
        review_date_to_append = [np.nan]
        ratings_to_append = [np.nan]
        reviews_to_append = [np.nan]   
        user_name_to_append = [np.nan]
    '''print(reviews_to_append)
    print(len(reviews_to_append))
    print(ratings_to_append)
    print(len(ratings_to_append))
    print(user_name_to_append)
    print(len(user_name_to_append))'''
    for h in range(len(reviews_to_append)):
        course_title.append(title_to_append)
        description.append(description_to_append)
        difficulty_level.append(diff_level_to_append)
        duration.append(duration_to_append)
        skills.append(skills_to_append)
        instructors.append(instructors_to_append)
        rating.append(ratings_to_append[h])
        review_date.append(review_date_to_append[h])
        user_name.append(user_name_to_append[h])
        reviews.append(reviews_to_append[h])
    i+=1

In [277]:
len(course_title)

258

In [278]:
d = {'course_title': course_title, 'difficulty_level': difficulty_level, 'duration': duration, 
    'skills': skills, 'instructors': instructors, 'description':description, 'rating':rating,
    'review_date': review_date, 'user_name': user_name, 'reviews':reviews}
new_df = pd.DataFrame(data=d)

In [280]:
# download data as csv
new_df.to_csv('/Users/lijiaqi/Documents/usc/2022fall/DSCI560/udacity_data.csv')

In [199]:
for i in range(len(duration)):
    if skills[i] == []:
        print(links[i])

https://www.udacity.com/course/aws-machine-learning-foundations--ud065
https://www.udacity.com/course/introduction-to-machine-learning-using-microsoft-azure--ud00333
https://www.udacity.com/course/ai-fundamentals--ud099
https://www.udacity.com/course/linear-algebra-refresher-course--ud953
https://www.udacity.com/course/machine-learning-unsupervised-learning--ud741
https://www.udacity.com/course/big-data-analytics-in-healthcare--ud758
https://www.udacity.com/course/intel-edge-AI-fundamentals-with-openvino--ud132
https://www.udacity.com/course/artificial-intelligence--ud954
https://www.udacity.com/course/secure-and-private-ai--ud185
https://www.udacity.com/course/model-building-and-validation--ud919
https://www.udacity.com/course/data-visualization-and-d3js--ud507
https://www.udacity.com/course/machine-learning-for-trading--ud501
https://www.udacity.com/course/machine-learning--ud262
https://www.udacity.com/course/intro-to-hadoop-and-mapreduce--ud617
https://www.udacity.com/course/real-t

In [ ]:
    # similar to diff_level, find all possible ways to find duration and append it/NO Duarion Specified to the list
    time_list = list(set([a.text for a in driver.find_elements(By.XPATH, "//div/div[@class='_16ni8zai m-b-0 m-t-1s']/span") if 'complete' in a.text]))
    if len(time_list) == 0:
        duration_need_to_append = np.nan
    else:
        duration_need_to_append = time_list[0]          
    # for all skills in course page, create a list and append to skills list
    # one course could have serveral skills
    List_skills_per_course = driver.find_elements(By.XPATH, "//li/span/span/span[@class='_ontdeqt']")
    skills_need_to_append = [a.text for a in List_skills_per_course]
    # similar to skills in course page, create a list of instructors in each course and append to instructors list
    # one course could have serveral instructors and some of them could not be name of instructor but certificate (such as google cetificate)
    List_instructors_per_course = driver.find_elements(By.XPATH, "//div/h3[@class='instructor-name headline-3-text bold']")
    instructors_need_to_append = [a.text for a in List_instructors_per_course]
    # one course has only one description but one description might be divided into multi parts in source code
    # so we create a list of descriptions in each course and paste them together appending to instructors list
    description_list = [a.text for a in driver.find_elements(By.XPATH, "//div/div/div[@class='content']/div[@class='content-inner']/p")]
    description_need_to_append = ' '.join(description_list)

In [177]:
len(links)

275

In [171]:
links_after = [a.split('--')[1] for a in links]
len(list(set(links_after)))

275

In [179]:
for i in course_title:
    if 'Artificial Intelligence' in i:
        print(i)

Artificial Intelligence for Trading
Artificial Intelligence
Artificial Intelligence
Intro to Artificial Intelligence
Artificial Intelligence for Robotics


In [178]:
links

['https://www.udacity.com/course/data-engineer-nanodegree--nd027',
 'https://www.udacity.com/course/product-manager-nanodegree--nd036',
 'https://www.udacity.com/course/data-analyst-nanodegree--nd002',
 'https://www.udacity.com/course/intro-to-programming-nanodegree--nd000',
 'https://www.udacity.com/course/data-scientist-nanodegree--nd025',
 'https://www.udacity.com/course/digital-marketing-nanodegree--nd018',
 'https://www.udacity.com/course/c-plus-plus-nanodegree--nd213',
 'https://www.udacity.com/course/business-analytics-nanodegree--nd098',
 'https://www.udacity.com/course/programming-for-data-science-nanodegree--nd104',
 'https://www.udacity.com/course/ux-designer-nanodegree--nd578',
 'https://www.udacity.com/course/deep-learning-nanodegree--nd101',
 'https://www.udacity.com/course/blockchain-developer-nanodegree--nd1309',
 'https://www.udacity.com/course/ai-programming-python-nanodegree--nd089',
 'https://www.udacity.com/course/react-nanodegree--nd019',
 'https://www.udacity.com

In [125]:
len(course_title)

272

# new way of web scraping

In [112]:
course_title = []
difficulty_level = []
duration = []
skills = []
instructors = []
description = []
rating = []
num_rating = []
like_percent = []
offered_by = []
language = []
syllabus_heading = []
syllabus_details = []
#there are 84 pages of programs
'''main_url = "https://www.coursera.org/courses?page=2&index=prod_all_products_term_optimization"
driver.get(main_url)
links_in_main_url = driver.find_elements(By.XPATH, "//div[@class='css-1j8ushu']/div/a") # get url of all programs/certificate in main url of coursera
links_in_main_url = [elem.get_attribute('href') for elem in links_in_main_url] # change html to list
for link_for_prog in links_in_main_url: # click in each program url and find corresponding courses url to explore
    driver.get(link_for_prog)
    links_for_courses = driver.find_elements(By.XPATH, "//div/a[@data-e2e='course-link']")#.click()
    links_in_prog_url = [elem.get_attribute('href') for elem in links_for_courses]'''
main_url = 'https://www.coursera.org/sitemap~www~courses.xml'
driver.get(main_url)
source = requests.get(main_url).text
soup = BeautifulSoup(source, 'lxml')
links_in_prog_url = [site.text for site in soup.find_all('loc')] # list of links for courses

#for update can delete for submission
for index_link in range(len(links_in_prog_url)):
    #https://www.coursera.org/learn/googlecloud-4-in-a-row-game-development-with-google-cloud-unity-hfcwa diff format so null
    if links_in_prog_url[index_link] == "https://www.coursera.org/learn/towards-agenda-2030":
        link_till_now = links_in_prog_url[index_link]
        break
links_in_prog_url = links_in_prog_url[index_link:]
        
        
for course_link in links_in_prog_url: # click in each course url of each program to get useful information
    driver.get(course_link)
    course_title_find_element = driver.find_elements(By.XPATH, "//div/h1[@class='banner-title banner-title-without--subtitle m-b-0']")
    #banner-title banner-title-without--subtitle m-b-0
    if len(course_title_find_element) == 0:
        course_title_need_to_append = np.nan
    else:
        course_title_need_to_append = course_title_find_element[0].text
    # find all possible ways to find difficulty level and append it/NO Level Specified to the list 
    list_levels_a = list(set([a.text for a in driver.find_elements(By.XPATH, "//div/div[@class='_16ni8zai m-b-0 m-t-1s']") if 'Level' in a.text])) # one possible way to get diff level
    list_levels_b = list(set([b.text for b in driver.find_elements(By.XPATH, "//div/div[@class='_16ni8zai m-b-0']") if 'Level' in b.text])) # another possible way to get diff level
    diff_level_all_possible = list(set(list_levels_a + list_levels_b)) # find level related info and append in diff level
    if len(diff_level_all_possible) == 0:
        difficulty_level_need_to_append = np.nan
    else:
        difficulty_level_need_to_append = diff_level_all_possible[0]
    # similar to diff_level, find all possible ways to find duration and append it/NO Duarion Specified to the list
    time_list = list(set([a.text for a in driver.find_elements(By.XPATH, "//div/div[@class='_16ni8zai m-b-0 m-t-1s']/span") if 'complete' in a.text]))
    if len(time_list) == 0:
        duration_need_to_append = np.nan
    else:
        duration_need_to_append = time_list[0]          
    # for all skills in course page, create a list and append to skills list
    # one course could have serveral skills
    List_skills_per_course = driver.find_elements(By.XPATH, "//li/span/span/span[@class='_ontdeqt']")
    skills_need_to_append = [a.text for a in List_skills_per_course]
    # similar to skills in course page, create a list of instructors in each course and append to instructors list
    # one course could have serveral instructors and some of them could not be name of instructor but certificate (such as google cetificate)
    List_instructors_per_course = driver.find_elements(By.XPATH, "//div/h3[@class='instructor-name headline-3-text bold']")
    instructors_need_to_append = [a.text for a in List_instructors_per_course]
    # one course has only one description but one description might be divided into multi parts in source code
    # so we create a list of descriptions in each course and paste them together appending to instructors list
    description_list = [a.text for a in driver.find_elements(By.XPATH, "//div/div/div[@class='content']/div[@class='content-inner']/p")]
    description_need_to_append = ' '.join(description_list)
    # Find number of ratings
    # since we can have no rating for a courser, we fill in 0 instead
    try:
        num_rating_need_to_append = driver.find_element(By.XPATH, "//div/span[@data-test='ratings-count-without-asterisks']").text
    except NoSuchElementException:  #spelling error making this code not work as expected
        num_rating_need_to_append = np.nan # course has no info about num_rating, probably 0
    # find percent of likes
    try:
        like_percent_need_to_append = driver.find_element(By.XPATH, "//div/div[@class='_wmgtrl9 m-l-1s expertise-rating-item__score color-white']/span").text
    except NoSuchElementException:  #spelling error making this code not work as expected
        like_percent_need_to_append = np.nan # course has no info about it, probably 0
    try:
        rating_need_to_append = driver.find_element(By.XPATH, "//div/span[@data-test='number-star-rating']").text
    except NoSuchElementException:  #spelling error making this code not work as expected
        rating_need_to_append = np.nan # course has no info about it, probably 0
    try:
        offered_by_need_to_append = driver.find_element(By.XPATH, "//div/a/h3[@class='headline-4-text bold rc-Partner__title']").text
    except NoSuchElementException:  #spelling error making this code not work as expected
        offered_by_need_to_append = np.nan # course has no info about it, probably non/ self sponsored
    '''try:
        language_need_to_append = driver.find_elements(By.XPATH, "//div[@class='_1tu07i3a']/div[@class='_16ni8zai m-b-0']")
        if len(language_need_to_append) != 0:
            driver.find_elements(By.XPATH, "//div[@class='_1tu07i3a']/div[@class='_16ni8zai m-b-0']")[-1].get_attribute('textContent')
        else:
            try:
                language_need_to_append = driver.find_elements(By.XPATH, "//div[@class='cds-63 css-1147z5y cds-65 cds-grid-item']/span[@class='rc-TooltipWrapper css-0']/span[@class='cds-33 css-s6kthz cds-35']")[-2].get_attribute('textContent')
                if 'selenium' in language_need_to_append:
                    try:
                        language_need_to_append = driver.find_elements(By.XPATH, "//div/div[@class='div']/div[@class='_16ni8zai m-b-0']")[-1].get_attribute('textContent')
                    except NoSuchElementException: 
                        language_need_to_append = np.nan
                else:
                    language_need_to_append = np.nan
            except NoSuchElementException:
                language_need_to_append = np.nan # course has no info about it, probably non/ self sponsored              
    except NoSuchElementException:  #spelling error making this code not work as expected
        try:
            language_need_to_append = driver.find_elements(By.XPATH, "//div[@class='cds-63 css-1147z5y cds-65 cds-grid-item']/span[@class='rc-TooltipWrapper css-0']/span[@class='cds-33 css-s6kthz cds-35']")[-2].get_attribute('textContent')
        except NoSuchElementException:
            language_need_to_append = np.nan # course has no info about it, probably non/ self sponsored  '''
    try:
        language_need_to_append = driver.find_elements(By.XPATH, "//div/div[@class='_1tu07i3a']/div[@class='_16ni8zai m-b-0']")[-1].get_attribute('textContent')
    except NoSuchElementException: 
        language_need_to_append = np.nan
 


    try:
        syllabus_heading_need_to_append_list = driver.find_elements(By.XPATH, "//div[@class='SyllabusModule']/h3[@class='headline-2-text bold m-b-2 ']")
        syllabus_heading_need_to_append = [a.get_attribute('textContent') for a in syllabus_heading_need_to_append_list]
    except NoSuchElementException:  #spelling error making this code not work as expected
        syllabus_heading_need_to_append = [np.nan] # course has no info about it, probably non/ self sponsored  
    try:
        syllabus_details_need_to_append_list = driver.find_elements(By.XPATH, "//div[@class='SyllabusModule']/div/div/div/div[@class='content-inner']/p")
        syllabus_details_need_to_append = [a.get_attribute('textContent') for a in syllabus_details_need_to_append_list]
    except NoSuchElementException:  #spelling error making this code not work as expected
        syllabus_details_need_to_append = [np.nan] # course has no info about it, probably non/ self sponsored       
 
        
        
        
        
        
        
        
        
        
        
        
        
    course_title.append(course_title_need_to_append)
    difficulty_level.append(difficulty_level_need_to_append)
    duration.append(duration_need_to_append)
    skills.append(skills_need_to_append)
    instructors.append(instructors_need_to_append)
    description.append(description_need_to_append)
    num_rating.append(num_rating_need_to_append)
    like_percent.append(like_percent_need_to_append)
    rating.append(rating_need_to_append)
    offered_by.append(offered_by_need_to_append)
    language.append(language_need_to_append)
    syllabus_heading.append(syllabus_heading_need_to_append)
    syllabus_details.append(syllabus_details_need_to_append)
    '''if len(course_title) != len(reviews):
        print(course_link)'''
        #print(reviews_list.text)






In [113]:
d = {'course_title': course_title, 'difficulty_level': difficulty_level, 'duration': duration, 
    'skills': skills, 'instructors': instructors, 'description':description, 'rating':rating,
    'offered_by': offered_by, 'language': language, 'syllabus_heading':syllabus_heading,
    'syllabus_details':syllabus_details}
Coursera_df = pd.DataFrame(data=d)
Coursera_df.to_csv('/Users/lijiaqi/Documents/usc/2022fall/DSCI560/coursera_data_course_level_update12.csv')

In [77]:
#delete 1st line in update1
Coursera_df

,course_title,difficulty_level,duration,skills,instructors,description,rating,offered_by,language,syllabus_heading,syllabus_details
0,"Cloud Computing Concepts, Part 1",Beginner Level,Approx. 23 hours to complete,"[Distributed Algorithm, Distributed Computing,...",[Indranil Gupta],"Cloud computing systems today, whether open-so...",4.4\nstars,University of Illinois at Urbana-Champaign,English,"[Week 1: Orientation, Introduction to Clouds, ...",[This course is oriented towards learners with...
1,The Challenges of Global Health,Beginner Level,Approx. 20 hours to complete,[],[Dr. David Boyd],What are the world’s major health challenges? ...,4.8\nstars,Duke University,English,"[Getting Started, Basic Concepts in Global Hea...","[Start here! , This module covers some of the ..."
2,Digital Marketing Analytics in Theory,Beginner Level,Approx. 20 hours to complete,"[Digital Marketing, Data Collection, Analytics...",[Kevin Hartman],Successfully marketing brands today requires a...,4.5\nstars,University of Illinois at Urbana-Champaign,English,[Course Overview and The Day The Geeks Took Ov...,"[In the orientation, you will become familiar ..."
3,Digital Product Management: Modern Fundamentals,Beginner Level,Approx. 11 hours to complete,"[Product/Market Fit, Product Management, Desig...",[Alex Cowan],"Not so long ago, the job of product manager wa...",4.7\nstars,University of Virginia,English,"[Achieving Focus and #Winning, Using Today's P...","[As a product manager, you and your team will ..."
4,The Nature of Genius,Beginner Level,Approx. 28 hours to complete,"[Creativity, Art, History, Psychology, Art His...",[Craig Wright\nTOP INSTRUCTOR],"Today ""genius"" is all around us. Celebrities, ...",NaN,Yale University,English,"[Genius Defined, Genius Denied , Genius and...","[In Section 1 of Module 1 we determine, joking..."
...,...,...,...,...,...,...,...,...,...,...,...
1047,Introduction to Cloud Identity,Beginner Level,Approx. 11 hours to complete,"[Identity Management (IDM), Google, Security A...",[Google Cloud Training],Introduction to Cloud Identity serves as the s...,4.6\nstars,Google Cloud,English,"[Summary, Use Case Example, & Resources]",[This module serves as the summary of your tra...
1048,Reliable Google Cloud Infrastructure: Design a...,Advanced Level,Approx. 8 hours to complete,"[Disaster Recovery, Site Reliability Engineeri...",[Google Cloud Training],This course equips students to build highly re...,4.7\nstars,Google Cloud,English,"[Google Cloud and Hybrid Network Architecture,...","[ In this module, we discuss Google Cloud netw..."
1049,Reliable Cloud Infrastructure: Design and Proc...,Intermediate Level,Approx. 8 hours to complete,[],[Google Cloud Training],Este curso prepara estudantes para criar soluç...,4.7\nstars,Google Cloud,Portuguese (Brazilian),"[Introdução, Definição de serviços, Design e a...","[É um prazer ter sua presença no curso ""Reliab..."
1050,Reliable Cloud Infrastructure: Design and Proc...,Intermediate Level,Approx. 8 hours to complete,[],[Google Cloud Training],"En este curso, los alumnos aprenderán a crear ...",4.9\nstars,Google Cloud,Spanish,"[Introducción, Definición de servicios, Diseño...",[Le damos la bienvenida al curso Reliable Goog...


# Udacity data in course level

In [ ]:
elems = driver.find_elements(By.XPATH,"//a[@href]")
links = []
for elem in elems:
    if '--' in elem.get_attribute("href"):
        if elem.get_attribute("href") not in links:
            links.append(elem.get_attribute("href"))
links = links[:-3]

In [ ]:
# if missing then nan
course_title = [] #
difficulty_level = [] #
duration = [] #
skills = [] #
instructors = [] #
description = [] #
rating = []
num_rating = []
like_percent = []
offered_by = []
language = []
syllabus_heading = []
syllabus_details = []
main_url = 'https://www.udacity.com/courses/all'
source = driver.get(main_url)
source_string = driver.page_source
soup = BeautifulSoup(source_string, 'lxml')
all_courses = soup.find_all(class_ = 'card_container__25DrK')
# find all links corresponding to course_titles in order to find reviews
'''elems = driver.find_elements(By.XPATH,"//a[@href]")
links = []
for elem in elems:
    if '--' in elem.get_attribute("href"):
        if elem.get_attribute("href") not in links:
            links.append(elem.get_attribute("href"))
links = links[:-3]'''
i = 0 
for course in all_courses: 
    title_to_append = course.find(class_ = 'card_title__35G97').text
    description_to_append = course.find(class_ = 'card_summary__1HlQ7').text
    candidate_level_intermediate = course.find(class_ = 'card_level__2HNxe card_intermediate__prUKS')
    candidate_level_beginner = course.find(class_ = 'card_level__2HNxe card_beginner__1tLPm')
    candidate_level_advanced = course.find(class_ = 'card_level__2HNxe card_advanced__3oCw3')
    if candidate_level_intermediate:
        diff_level_to_append = candidate_level_intermediate.text
    elif candidate_level_beginner:
        diff_level_to_append = candidate_level_beginner.text
    elif candidate_level_advanced:
        diff_level_to_append = candidate_level_advanced.text
    else:
        diff_level_to_append = np.nan
    course_review_url = links[i]
    review_source = driver.get(course_review_url)
    try: #scrape durations
        duration_to_append = driver.find_element(By.XPATH, "//ul[@class='degree-info-columns_columnList__D-UoB']/li/div/h5").text +  driver.find_element(By.XPATH, "//ul[@class='degree-info-columns_columnList__D-UoB']/li/div/p").text
    except NoSuchElementException:  #spelling error making this code not work as expected
        duration_to_append = np.nan
    try: #scrape skills
        List_skills_per_course = driver.find_elements(By.XPATH, "//div/ol/li[@class='degree-syllabus_part__hQ0du']/h5")
        skills_to_append = [a.text for a in List_skills_per_course]    
    except NoSuchElementException:
        try:
            List_skills_per_course = driver.find_elements(By.XPATH, "//a/div/h2[@class='h4']")
            skills_to_append = [a.text for a in List_skills_per_course]
        except NoSuchElementException:
            skills_to_append = [np.nan]
    try:  #scrape instructors      
        instructors_per_course = driver.find_elements(By.XPATH, "//li[@class='card_container__uEUZV card_topRight__1i-5a _brand-refresh_card__NfYin']/article/h3")
        instructors_to_append = [a.text for a in instructors_per_course]
    except NoSuchElementException:
        instructors_to_append = [np.nan]
    try:
        rating_to_append = driver.find_element(By.XPATH, "//div/b[@class='_header_statsAverageRating__2qC_-']").text 
    except NoSuchElementException:
        rating_to_append = np.nan
    try:
        num_rating_need_to_append = driver.find_element(By.XPATH, "//div/span[@class='_header_statsRatingCount__KmOE7']").text
    except NoSuchElementException:  #spelling error making this code not work as expected
        num_rating_need_to_append = np.nan # course has no info about num_rating, probably 0    
    likes_need_to_append = np.nan
    offered_by_need_to_append = np.nan
    language_need_to_append = np.nan # English for missing since most of classes in Udacity is in English
    syllabus_heading_need_to_append = [np.nan]
    syllabus_details_need_to_append = [np.nan]
        
        
        
        
    '''print(reviews_to_append)
    print(len(reviews_to_append))
    print(ratings_to_append)
    print(len(ratings_to_append))
    print(user_name_to_append)
    print(len(user_name_to_append))'''

    course_title.append(title_to_append)
    description.append(description_to_append)
    difficulty_level.append(diff_level_to_append)
    duration.append(duration_to_append)
    skills.append(skills_to_append)
    instructors.append(instructors_to_append)
    rating.append(ratings_to_append)
    num_rating.append(num_rating_need_to_append)
    like_percent.append(like_percent_need_to_append)
    offered_by.append(offered_by_need_to_append)
    language.append(language_need_to_append)
    syllabus_heading.append(syllabus_heading_need_to_append)
    syllabus_details.append(syllabus_details_need_to_append)    
 
    i+=1

In [ ]:
d = {'course_title': course_title, 'difficulty_level': difficulty_level, 'duration': duration, 
    'skills': skills, 'instructors': instructors, 'description':description, 'rating':rating,
    'offered_by': offered_by, 'language': language, 'syllabus_heading':syllabus_heading,
    'syllabus_details',syllabus_details}
Coursera_df = pd.DataFrame(data=d)
Coursera_df.to_csv('/Users/lijiaqi/Documents/usc/2022fall/DSCI560/coursera_data_course_level.csv')

In [12]:
course_title#_need_to_append
#difficulty_level
'''duration 
skills 
instructors
description
num_rating
like_percent
rating
offered_by'''

'duration \nskills \ninstructors\ndescription\nnum_rating'

In [60]:
syllabus_heading

[[], [], [], []]

In [61]:
a = [np.nan]
print(a)

[nan]
